<a href="https://colab.research.google.com/github/jcabarcau/Me/blob/main/MadGraphEventsFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Actividad: Organización de Datos y obtención de Gráficas de variables invariantes a partir de Simulaciones de MadGraph.**

In [ ]:
#@author: jcabarcau

Importamos nuestras librerías:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import plotly.express as px

# **Procesamiento del archivo de salida de MadGraph en formato LHE usando Bash.**

Usamos bash/shell para extraer los renglones que nos interesan (7, 8, 9 y 10) del archivo "unweighted_Events.lhe" ubicado en nuestro Drive y los guardamos en un nuevo archivo llamado "eventos_pxpypzE.csv", el cuál se guardará temporalmente, mientras sea utilizado en el Colab.

In [ ]:
%shell
!cat /content/unweighted_events.lhe | grep "13  " | awk '{print $7", "$8", "$9", "$10}' > /content/eventos_pxpypzE.csv

Mostramos la parte superior (header/encabezado) de nuestro nuevo dataset filtrado:

In [ ]:
events=pd.read_csv("/content/eventos_pxpypzE.csv",header=None,delimiter=',',dtype=np.float64) 
events.head()

Cada columna representa los momentos $p_x$, $p_y$, $p_z$, y la energía $E$ de cada partícula.

# **Obtención de los histogramas del momento transverso y del momento total.**

Obtenemos los histogramas del momento transverso y del momento total, a partir de los datos de los momentos $p_x$, $p_y$ y $p_z$.

In [ ]:
momento_transverso = np.sqrt(events[0]**2+events[1]**2)
momento_transverso = pd.DataFrame(momento_transverso)
histograma_momento_transverso = px.histogram(momento_transverso, x = momento_transverso[0], title = 'Momento transverso')
histograma_momento_transverso.show()

In [ ]:
momento_total = np.sqrt(events[0]**2+events[1]**2+events[2]**2)
momento_total = pd.DataFrame(momento_total)
histograma_momento_total = px.histogram(momento_total, x = momento_total[0], title = 'Momento total')
histograma_momento_total.show()

# **Creación de 4-vectores para su posterior manipulación.**

Separamos los pares de partículas en 2 grupos para luego juntarlas y representarlas en una misma columna, separadas por filas.

In [ ]:
events_0 = events.iloc[lambda x: x.index % 2 == 0]
events_0 = events_0.reset_index(drop = True)

events_1 = events.iloc[lambda x: x.index % 2 != 0]
events_1 = events_1.reset_index(drop = True)


events_0.rename(columns = {events_0.columns[0]: 'px1',
                          events_0.columns[1]: 'py1',
                          events_0.columns[2]: 'pz1',
                          events_0.columns[3]: 'E1'}, inplace=True)

events_1.rename(columns = {events_1.columns[0]: 'px2',
                           events_1.columns[1]: 'py2',
                           events_1.columns[2]: 'pz2',
                           events_1.columns[3]: 'E2'}, inplace=True)

datos = events_0.join(events_1)
datos.head()

Guardamos nuetro nuevo dataset en un nuevo archivo para su posterior manipulación.

In [ ]:
datos.to_csv('/content/mumu')

# **Obtención del histograma de la masa invariante.**

Para obtener la masa invariante, primero sumamos ambos cuadrimomentos y sacamos su magnitud, recordando que en el espacio de Minkowski la masa invariante, en unidades naturales, está dada por

$$m_0=\sqrt{E^2-p_x^2-p_y^2-p_z^2}$$


Para sacar la masa invariante de ambos muones, requerimos primero sumar sus cuadrimomentos componente a componente y sacar la magnitud, es decir

$$M = \sqrt{(E_1+E_2)^2-(p_{x_1}+p_{x_2})^2-(p_{y_1}+p_{y_2})^2-(p_{z_1}+p_{z_2})^2}$$ 

In [ ]:
masa_invariante = np.sqrt( (datos['E1']+datos['E2'])**2
                  -(datos['px1']+datos['px2']**2)
                  -(datos['py1']+datos['py2']**2)
                  -(datos['pz1']+datos['pz2'])**2)
masa_invariante.head()

masa_invariante = pd.DataFrame(masa_invariante)
histograma_masa_invariante = px.histogram(masa_invariante, x = masa_invariante[0], title = 'Masa Invariante')
histograma_masa_invariante.show()

# **Distribución de $\phi$.**

Obtenemos los valores de $\phi_1$ y $\phi_2$ usando la ecuación $$\phi = \arctan\Big(\frac{p_y}{p_x}\Big)$$

y realizamos los histogramas correspondientes.

In [ ]:
phi_1 = arctan(datos['py1'], datos['px1'])
phi_2 = arctan(datos['py2'], datos['px2'])

histograma_phi1=px.histogram(phi_1,x=phi_1[:],nbins=100)
histograma_phi1.update_layout(
    title=r"$ \text{ Distribucion de }  \phi_1$",
    xaxis_title="$\phi_1 \t \ {(en \ radianes)}$",
    yaxis_title="Núm. de Eventos")
histograma_phi1.show()

histograma_phi2=px.histogram(phi_2,x=phi_2[:],nbins=100)
histograma_phi2.update_layout(
    title=r"$ \text{ Distribucion de }  \phi_2$",
    xaxis_title="$\phi_2 \t \ {(en \ radianes)}$",
    yaxis_title="Núm. de Eventos")
histograma_phi2.show()


# **Obtención de $\Delta \phi$**

Obtenemos la Distribución de $\Delta \phi$, con la ecuación $\Delta \phi = |\phi_2 - \phi_1 |$, la cuál en teoría debería darnos como resultado que $\Delta \phi = \pi$ rads.

In [ ]:
dphi = np.abs(phi_2-phi_1)

delta_phi=px.line(dphi,x=dphi[:])
delta_phi.update_layout(
    title=r"$ \text{ Distribucion de }  d\phi$",
    xaxis_title="$\Delta \phi \t \ {(en \ radianes)}$",
    yaxis_title="Núm. de Eventos")
delta_phi.show()

Por lo tanto $\Delta \phi = \pi$ rads. en todos los eventos, confirma que el procedimiento fue el correcto.

# **Agradecimientos**

Expreso mi total agradecimiento a la Dra. Isabel Pedraza y a mi compañero de trabajo, Alan Covarrubias por sus aportes, herramientas y enseñanzas dadas para el aprendizaje y uso de Python y Google Colab.